#### R in a nutshell

- Statistical programming environments
- Originally designed and implemented by statisticians
- Widely popular due to its extensive collection of community-contributed packages
- Quickly gaining places among traditional proprietary tools such as SAS and STATA for data analytics

#### Learning Objectives

- Understand basic programming concepts: variables, assignment, functions, loops, conditions
- Understand core R concepts: data loading, data types, data access, libraries
- Understand advanced R concepts: data manipulation, visualization
- Understand HPC concepts: running R codes on the Palmetto supercomputer via batch submission scripts

#### Materials on this notebook is based on two lessons by Software Carpentry and Data Carpentry:

- Introduction to Programming using R
- Data Analysis and Visualization in R for Ecology

## Where am I?

In [ ]:
getwd()

In [ ]:
surveys <- read.csv("data/combined.csv")
head(surveys)

## Data Frames Manipulation

In [ ]:
if (!require('dplyr', character.only = TRUE)){
  install.packages('dplyr', dep = TRUE)
  if (!require('dplyr', character.only = TRUE)){
    print ('Package not found')
  }
}

** Common `dplyr` functions: **
- `select()`
- `filter()`
- `mutate()`
- `groupby()`
- `summarize()`
- `%>%`

** Cheatsheet: **
http://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf

** Selecting columns and filtering rows **

In [ ]:
select(surveys, plot_id, species_id, weight)

In [ ]:
filter(surveys, year == 1995)

** Pipes: combining multile select and filter actions **

In [ ]:
surveys %>%
  filter(weight < 5) %>%
  select(species_id, sex, weight)

In [ ]:
surveys_sml <- surveys %>%
  filter(weight < 5) %>%
  select(species_id, sex, weight)
surveys_sml

** Challenge: **

Using pipes, subset the `survey` data to include individuals collected before 1995 and retain only the columns `year`, `sex`, and `weight`. 

** Mutate: create new columns based on existing columns **

In [ ]:
surveys %>%
  mutate(weight_kg = weight / 1000) %>%
  head  

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  mutate(weight_kg = weight / 1000) %>%
  head

- `is.na`: determines whether something is NA (not available - missing values)
- `!`: negates a logical value

** Challenge: **

Create a new data frame from the survey data that meets the following criteria: 

contains only the species_id column and a new column called hindfoot_half containing values that are half the hindfoot_length values. In this hindfoot_half column, there are no NAs and all values are less than 30.

** Split-apply-combine data analysis and the summarize() function **

- split data into groups
- apply some analysis to each group
- combine the results

** `group_by()` and `summarize()`: **

In [ ]:
surveys %>%
  group_by(sex) %>%
  summarize(mean_weight = mean(weight, na.rm = TRUE))

In [ ]:
surveys %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight, na.rm = TRUE))

- `NaN`: not a number
- Need filtering to remove missing values

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight))

In [ ]:
x <- surveys %>%
       filter(!is.na(weight)) %>%
       group_by(sex, species_id) %>%
       summarize(mean_weight = mean(weight))
str(x)

- The `tbl_data` structure is very similar to a data frame; for our purposes the only difference is that, in addition to displaying the data type of each column under its name, it only prints the first few rows of data and only as many columns as fit on one screen.
- If you want to display more data, you use the `print()` function at the end of your chain with the argument `n` specifying the number of rows to display:

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight)) %>%
  print(n = 15)

Summarization on multiple variables at the same time is also possible

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight),
            min_weight = min(weight))

** Tallying: simply counting things: **

In [ ]:
surveys %>%
  group_by(sex) %>%
  tally

** Challenge: **

- How many individuals were caught in each plot_type surveyed?
- Use `group_by()` and `summarize()` to find the mean, min, and max hindfoot length for each species (using `species_id`).

** Exporting data to file: **

- Write cleaned data to file, so that data cleaning process does not have to redone
- Output data should be stored in different location from original raw data

Conditional statement:


if (condition is true){

    do something
    
} else {

    do something else
    
}

In [ ]:
current_dir <- getwd()
output_data_dir <- 'output_data'

if (!file.exists(output_data_dir)){
    dir.create(file.path(current_dir, output_data_dir))
} else {
    print ("Directory already exists")
}

In [ ]:
?dir.create

In [ ]:
?file.path

*Create a clean data set with missing observations removed*

In [ ]:
surveys_complete <- surveys %>%
  filter(species_id != "",         # remove missing species_id
         !is.na(weight),           # remove missing weight
         !is.na(hindfoot_length),  # remove missing hindfoot_length
         sex != "")                # remove missing sex

Additional filters: Remove rare species (less than 50 observations)

- Create index of rare species
- Filter rare species from the cleaned data set

In [ ]:
## Extract the most common species_id
species_counts <- surveys_complete %>%
  group_by(species_id) %>%
  tally %>%
  filter(n >= 50)

In [ ]:
## Only keep the most common species
surveys_complete <- surveys_complete %>%
  filter(species_id %in% species_counts$species_id)

*Write data to file*

In [ ]:
write.csv(surveys_complete, file = file.path(output_data_dir, "surveys_complete.csv"),
          row.names=FALSE)